In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 43.16 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [2]:
# creating dictonary with keys as expression and value as list of expression
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f) # reader object reads data row by row
  next(reader) # to skip column headers
  for row in reader:
    key = row[2].lower()
    # add value (image) to corresponding key
    if key in data:
      data[key].append(row[1])
    # make key value pair if not present
    else:
      data[key] = [row[1]]

In [3]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [4]:
# creating training and testing directory
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [5]:
# creating directories for each emotion in training and testing
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [6]:
# copying images to created directories
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  # splitting images into train and test
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [23]:
# model
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 49, 49, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 10, 10, 64)       

In [22]:
# generate array from images, generate lables from directory name and to 1 hot encode them
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 64
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 64
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [17]:
# early stopping model if acc is not improving in next 2 epochs
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [24]:
model.fit(train_generator,
          epochs = 10,
          verbose = 1,
          validation_data = test_generator,
          callbacks = [es])

Epoch 1/10
171/171 [==============================] - 19s 101ms/step - loss: 1.2066 - acc: 0.4947 - val_loss: 1.0655 - val_acc: 0.5040
Epoch 2/10
171/171 [==============================] - 16s 93ms/step - loss: 0.8998 - acc: 0.6552 - val_loss: 1.0457 - val_acc: 0.5999
Epoch 3/10
171/171 [==============================] - 16s 95ms/step - loss: 0.7960 - acc: 0.7171 - val_loss: 0.9256 - val_acc: 0.6608
Epoch 4/10
171/171 [==============================] - 16s 94ms/step - loss: 0.7629 - acc: 0.7311 - val_loss: 0.9256 - val_acc: 0.6754
Epoch 5/10
171/171 [==============================] - 18s 107ms/step - loss: 0.7166 - acc: 0.7544 - val_loss: 1.0795 - val_acc: 0.6255
Epoch 6/10
171/171 [==============================] - 16s 94ms/step - loss: 0.7134 - acc: 0.7551 - val_loss: 0.9970 - val_acc: 0.6652
